# Analyzing Used Car Listings

This project uses a dataset from *eBay Kleinanzeigen*, the German classifieds section from *ebay.de*, and includes a sampled subsection of over 50,000 datapoints collected for used car listings. These data are in the form of a dictionary, and include post information such as the date the post information was collected, the name of the car, the seller, the listing price, details about the car including transmission type, kilometers on the car's odometer, fueltype, as well as post-specific information such as the number of pictures included, date of post creation, etc.

The primary purpose is to familiarize the code-author with cleaning and analyze data using pandas in Jupyter, and thus does not have key business objectives clearly outlined at the onset.

#### Import pandas and NumPy libraries

In [1]:
import pandas as pd
import numpy as np

#### Read dataset and investigate general features of data

In [2]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')
autos.info()
autos.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


This dataset consists of 20 columns of information for 50,000 posts. The majority of columns have complete data, but `vehicleType`, `gearbox`, `model`, `fuelType`, and `notRepairedDamage` all have varying amounts of null data included, with the most being approximately 20% of the total entries.

## Data Cleaning

#### Format column names;

* Convert all names from "camelCase" to "snake_case"

* Rename several columns to improve interpretability

In [3]:
autos.columns = ['date_crawled','name','seller','offer_type','price',
                 'ab_test','vehicle_type','registration_year','gearbox',
                 'power_ps','model','odometer','registration_month',
                 'fuel_type','brand','unrepaired_damage','ad_created',
                 'num_photos','postal_code','last_seen']
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


#### Descriptive Statistics

In [4]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_photos,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-11 22:38:16,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Columns that could be excluded from analysis;

* `seller` and `offer_type`; all but one entry for both are identical

* `num_photos`; all entries are `0 / NaN`. The tool used to scrape these data may not have scraped images, or improperly counted the number of photos in posts.

In [5]:
autos.drop(['seller','offer_type','num_photos'], axis=1)

,date_crawled,name,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2016-03-27 14:38:19,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,"$24,900",control,limousine,2011,automatik,239,q5,"100,000km",1,diesel,audi,nein,2016-03-27 00:00:00,82131,2016-04-01 13:47:40
49996,2016-03-28 10:50:25,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,"$1,980",control,cabrio,1996,manuell,75,astra,"150,000km",5,benzin,opel,nein,2016-03-28 00:00:00,44807,2016-04-02 14:18:02
49997,2016-04-02 14:44:48,Fiat_500_C_1.2_Dualogic_Lounge,"$13,200",test,cabrio,2014,automatik,69,500,"5,000km",11,benzin,fiat,nein,2016-04-02 00:00:00,73430,2016-04-04 11:47:27
49998,2016-03-08 19:25:42,Audi_A3_2.0_TDI_Sportback_Ambition,"$22,900",control,kombi,2013,manuell,150,a3,"40,000km",11,diesel,audi,nein,2016-03-08 00:00:00,35683,2016-04-05 16:45:07


* Reformat `price` and `odometer` columns to numeric values by removing characters and converting subsequent value

In [6]:
autos['price'] = (autos['price']
                  .str.replace('$','')
                  .str.replace(',','')
                  .astype(int))

autos['odometer'] = (autos['odometer']
                    .str.replace('km','')
                    .str.replace(',','')
                    .astype(int))
autos.rename({'odometer':'odometer_km'}, axis=1, inplace=True)

### Investigate `odometer_km` and `price` data

Observe unique values, descriptive statistics, distribution, etc.

In [7]:
print('Descriptive statistics')
print(autos['odometer_km'].describe())
print('\n','Unique value counts for \'odometer_km\' value')
print(autos['odometer_km'].value_counts())

Descriptive statistics
count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

 Unique value counts for 'odometer_km' value
150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer_km, dtype: int64


The `odometer_km` appears to be rounded into common values, suggested posters on eBay.de were made to select from pre-defined values when reporting the odometer_km value.

While the reported mean is lower than 150,000km for this sample, it is likely that one pre-defined selection was "150,000+ km" and therefore the true mean odometer_km value would be higher, as cars above this value would be included in this bracket. This is supported by the "150,000km" value including more than half of the total dataset.

In [8]:
print('Descriptive statistics')
print(autos['price'].describe())
print('\n','20 most common value counts for \'price\' value')
print(autos['price'].value_counts().head(20))

Descriptive statistics
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

 20 most common value counts for 'price' value
0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
999      434
750      433
900      420
650      419
850      410
700      395
4500     394
300      384
2200     382
950      379
Name: price, dtype: int64


The mean `price` for these data, 9840 EUR, is significantly higher than the median 50% `price`, 2950 EUR. This suggests that these data are skewed by high-priced vehicles, particularly given the max `price` for any listing is 100,000,000 EUR. Additionally, 1,421 listings have a `price` of 0 EUR, which could be an error on the posting or the vehicles may have been listed as free.

The most common values observed are also generally rounded to a tens digit, with the exception of 999 EUR. It does not appear that `price` is grouped into the same pre-defined ranges that `odometer_km` values were, but rather that people may generally prefer to round to the tens digit for their listed `price`.

#### Removing price outliers

Given these observations, we will remove free / 0 EUR priced vehicles as well as any listings with a `price` above 350,000 EUR.

In [9]:
autos = autos[autos['price'].between(1,351000)]
autos['price'].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

Following removal of `price` outliers, it appears we retained ~48,500 listings, or 97% of the total dataset.

## Exploring the date columns

- `date_crawled`: reference of initial data collection by crawler.

- `last_seen`: reference of final time listing was still online.

- `ad_created`: when the initial listing was made on eBay.de.

- `registration_month`: month of vehicle title registration.

- `registration_year`: year of vehicle title registration.

`date_crawled`, `last_seen`, and `ad_created` are all currently `str` values and need to be converted to numeric date-time format.

In [10]:
autos[['date_crawled','last_seen','ad_created']][0:5]

,date_crawled,last_seen,ad_created
0,2016-03-26 17:47:46,2016-04-06 06:45:54,2016-03-26 00:00:00
1,2016-04-04 13:38:56,2016-04-06 14:45:08,2016-04-04 00:00:00
2,2016-03-26 18:57:24,2016-04-06 20:15:37,2016-03-26 00:00:00
3,2016-03-12 16:58:10,2016-03-15 03:16:28,2016-03-12 00:00:00
4,2016-04-01 14:38:50,2016-04-01 14:38:50,2016-04-01 00:00:00


Each string follows the `YYYY-MM-DD HH:MM:SS` format, but `ad_created` does not appear to contain any useful `HH:MM:SS` time data. Additionally, even though the crawler collected time data for these postings, it may not be relevant given these auctions generally last for several days or longer.

We will convert these columns to include only the relevant `YYYY-MM-DD` information in a `python` compliant date-time format and investigate the date distributions of these posts.

#### `date_crawled` information

In [11]:
(autos['date_crawled']
.str[:10]
.value_counts(normalize=True, dropna=False)
.sort_index())

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

Given these data, it looks like the web crawler took in posting information at roughly equal daily intervals between March 5th, 2016 and April 7th, 2016.

#### `last_seen` information

In [12]:
(autos['last_seen']
 .str[:10]
 .value_counts(normalize=True,dropna=False)
 .sort_index())

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

Here it appears most postings were no longer listed online for the crawler to collect data on for the same amount of time; the high proportion of `last_seen` at the end of the crawling period is likely explained by the data collection period ending rather than nearly 50% of used car listings selling over a 3 day period. An unlikely alternative could be if eBay.de had a "flash sale" promotion during these 3 days that resulted in major discounts to customers purchasing used cars, but without additional evidence supporting this, it is likely that the high proportion of `last_seen` data falling at the end of the crawling period is a function of the measure.

#### `ad_created` information

In [13]:
print(autos['ad_created'].str[:10].unique().shape)
print('\n','Unique \'ad_created\' values and proportion of listings by date')
print((autos['ad_created']
 .str[:10]
 .value_counts(normalize=True,dropna=False)
 .sort_index()))
print('\n','Most common 10 \'ad_created\' dates')
print(autos['ad_created']
 .str[:10]
 .value_counts(normalize=True,dropna=False)
 .head(10))
print('\n','Least common 10 \'ad_created\' dates')
print(autos['ad_created']
 .str[:10]
 .value_counts(normalize=True,dropna=False)
 .tail(10))

(76,)

 Unique 'ad_created' values and proportion of listings by date
2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038855
2016-04-04    0.036858
2016-04-05    0.011819
2016-04-06    0.003253
2016-04-07    0.001256
Name: ad_created, Length: 76, dtype: float64

 Most common 10 'ad_created' dates
2016-04-03    0.038855
2016-03-20    0.037949
2016-03-21    0.037579
2016-04-04    0.036858
2016-03-12    0.036755
2016-03-14    0.035190
2016-04-02    0.035149
2016-03-28    0.034984
2016-03-07    0.034737
2016-03-29    0.034037
Name: ad_created, dtype: float64

 Least common 10 'ad_created' dates
2016-01-14    0.000021
2016-02-09    0.000021
2016-02-17    0.000021
2016-01-29    0.000021
2016-01-03    0.000021
2016-02-08    0.000021
2015-06-11    0.000021
2016-02-22    0.000021
2016-01-16    0.000021
2016-01-07    0.000021
Name: ad_created, dtype: float64


There are over 76 unique values for the `ad_created` column, indicating there is a wide range of data for this measure. The 10 most common dates for `ad_created` values are all within the month that the crawler was active, suggesting that car listings are being created frequently and consistently.

The 10 least common `ad_created` dates have a range of dates, including from over one year prior to the crawler dates and some only one month prior. It may be that listings that do not sell within 1-2 months generally stay up, *or* that these less common listings older than 1 month have undesirable features to the listing that prevent the auction from being completed, possibly such as a high price, undesirable vehicle, or other features.

#### `registration_year` information

In [14]:
autos['registration_year'].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

Some values listed for `registration_year` are not possible values; the minimum year is `1000`, before cars were invented, while the maximum value is `9999`, well into the foreseeable future.

These values and others may represent missing or incomplete data, particularly given that `9999` is often used to indicate `NAN` while keeping the input values as type `int`.

##### Recoding incorrect `registration_year` values with available information

The maximum value for `registration_year` is `2016`, the same year the crawler collected these data, and a listed car could not have been registered after this year.

The minimum value for `registration_year` is slightly trickier. We will use the year that cars were first registered (`1912`) to broadly apply across the whole data.

If we wanted to be more specific and certain about these data, we could cross-reference the date that an auto manufacturer `brand` was established and set that year as the minimum value for each listing with that `brand` listed. If we needed even more certainty, we could additionally cross-reference the `model` value and set the year that the `brand` began producing the listed `model` as the minimum possible `registration_year` value for each listing.

In [15]:
autos = autos[autos['registration_year'].between(1912,2016)]
print(autos['registration_year'].describe())
print('\n','20 most common listed \'registration_year\' values')
print(autos['registration_year']
      .value_counts(normalize=True).head(20))

count    46676.000000
mean      2002.920709
std          7.120843
min       1927.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

 20 most common listed 'registration_year' values
2000    0.067615
2005    0.062902
1999    0.062066
2004    0.057910
2003    0.057824
2006    0.057203
2001    0.056474
2002    0.053261
1998    0.050626
2007    0.048783
2008    0.047455
2009    0.044670
1997    0.041799
2011    0.034772
2010    0.034043
1996    0.029416
2012    0.028066
1995    0.026288
2016    0.026138
2013    0.017204
Name: registration_year, dtype: float64


Following this filtering, we removed another ~2,000 posts with invalid `registration_year` data, retaining ~93% of the original dataset.

The majority of listed `registration_year` data was after `2003`, while only a quarter of posts were first registered before `1999`. These data suggest most eBay.de auto listings are for relatively modern cars, within the past 20 years.

## `price` by `brand` from available listings

Certain auto brands generally are valued higher by consumers, while other auto brands may be more common and thus have lower prices. Using these data, we can explore the mean `price` by `brand` in the available listings.

#### `brand`s present in data

In [16]:
autos['brand'].value_counts(normalize=True)

volkswagen        0.211286
bmw               0.110057
opel              0.107550
mercedes_benz     0.096474
audi              0.086576
ford              0.069907
renault           0.047133
peugeot           0.029844
fiat              0.025645
seat              0.018275
skoda             0.016411
nissan            0.015276
mazda             0.015190
smart             0.014161
citroen           0.014011
toyota            0.012705
hyundai           0.010027
sonstige_autos    0.009791
volvo             0.009148
mini              0.008763
mitsubishi        0.008227
honda             0.007841
kia               0.007070
alfa_romeo        0.006642
porsche           0.006127
suzuki            0.005935
chevrolet         0.005699
chrysler          0.003514
dacia             0.002635
daihatsu          0.002507
jeep              0.002271
subaru            0.002142
land_rover        0.002100
saab              0.001650
jaguar            0.001564
daewoo            0.001500
trabant           0.001371
r

Not surprisingly, the 5 most common `brand`s listed on eBay.de are German auto manufacturers, although the next 5 are from other nations;

| Rank | `brand` | Nationality |
| :- | :- | :- |
| 1 | `volkswagen` | Germany |
| 2 | `bmw` | Germany |
| 3 | `opel` | Germany |
| 4 | `mercedez_benz` | Germany |
| 5 | `audi` | Germany |
| 6 | `ford` | USA |
| 7 | `renault` | France |
| 8 | `peugeot` | France |
| 9 | `fiat` | Italy |
| 10 | `seat` | Spain |

Given the number of auto `brand`s in the world that could be available, we will restrict further analyses only to those that make up at least `1%` of the available eBay.de data.

##### Restricting analyses to `brand`s with >1% of total data

In [17]:
brand_counts = autos['brand'].value_counts(normalize=True)
common_brands = brand_counts[brand_counts > .01].index
print(common_brands)

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat', 'seat', 'skoda', 'nissan', 'mazda', 'smart',
       'citroen', 'toyota', 'hyundai'],
      dtype='object')


#### Mean `price` by `brand`

In [18]:
brand_mean_prices = {}

for brand in common_brands:
    brand_specific = autos[autos['brand'] == brand]
    mean_price = brand_specific['price'].mean()
    brand_mean_prices[brand] = int(mean_price)
    
print('Mean price (EUR) by brand;')
bmp_series = pd.Series(brand_mean_prices)
pd.DataFrame(bmp_series, columns = ['mean_price'])

Mean price (EUR) by brand;


,mean_price
volkswagen,5402
bmw,8332
opel,2976
mercedes_benz,8628
audi,9336
ford,3749
renault,2475
peugeot,3094
fiat,2813
seat,4397


The "luxury" `brand`s appear to maintain their value even in used car listings, with `bmw`, `mercedez_benz`, and `audi` maintaining higher mean `price` values in the available listings. `skoda`, an "entry-level luxury" `brand`, also somewhat maintains a higher value, although the comparative popularity of the other 3 luxury brands over `skoda` may drive listing `price` for used `skoda`s down somewhat.

Of the non-luxury brands, many of the Japanese `brand`s such as `nissan`, `mazda`, `toyota`, and `hyundai` also maintain relatively high `price` values in their posts. This may be indicative of reliability of these vehicles, higher import fees for these `brands` to the EU markets, or potentially lower `odometer_km` values for these vehicles. Further analysis could identify potential sources for these prices.

Additionally, `volkswagen` maintains a relatively high `price` for used vehicles on eBay.de, which may indicate the perception of reliability of these vehicles, or alternatively it may be that this `brand` has more newer vehicles sold frequenlty on eBay.de, but again, requires further analysis.

`opel`, `ford`, `renault`, `fiat`, and `smart` are the lowest cost options in this list, and may reflect the perceived value of these used vehicles.

To investigate one potential influence on `price`, we look for possible interactions between `odometer_km` and `brand` that may impact listed `price`.

#### Mileage (`odometer_km`) by `brand` and potential influence on `price`

In [19]:
brand_mean_mileage = {}

for brand in common_brands:
    brand_specific = autos[autos['brand'] == brand]
    mean_mileage = brand_specific['odometer_km'].mean()
    brand_mean_mileage[brand] = int(mean_mileage)
    
mean_mileage = pd.Series(brand_mean_mileage).sort_values(ascending=False)
mean_prices = pd.Series(brand_mean_prices).sort_values(ascending=False)

brand_mileage_price = pd.DataFrame(mean_mileage, columns = ['mean_mileage'])
brand_mileage_price['price'] = mean_prices
brand_mileage_price

,mean_mileage,price
bmw,132572,8332
mercedes_benz,130788,8628
opel,129311,2976
audi,129157,9336
volkswagen,128707,5402
renault,128127,2475
peugeot,127153,3094
mazda,124464,4112
ford,124266,3749
seat,121131,4397


These data suggest there's a range for `mean_mileage` of ~30,000km, but counterintuitively it doesn't appear to be negatively correlated with `price` for all `brand`s. Rather, `brand` appears to be the driving factor in `price`, with the "luxury" `brand`s also having the highest `price`, despite `bmw`, `mercedes_benz`, and `audi` all having some of the highest mileage by the `odometer_km` value.

Outside of these luxury `brand`s, there may be some slight correlation  between `mean_mileage` and `price`, in that we can see `renault` cars having some of the highest `odometer_km` values while also having the lowest `price`. `brand` then appears to matter more for mid-level and luxury cars, while `odometer_km` matters more for entry-level cars.